# Assignment #9: Final project

---


Sebastian Fallström & Karim Kanji  <br>
IA-20<br>
20.12.2023<br>

In [ ]:
# Imports
import random

Data


In [ ]:
# Data for the assignment problem
tasks = {
    'Make Coffee': {'barista_skills': 4, 'customer_service': 3},
    'Serve Customers': {'customer_service': 5, 'speed': 3},
    'Clean Tables': {'efficiency': 4, 'attention_to_detail': 3},
    'Manage Inventory': {'organization': 5, 'efficiency': 4},
    'Bake Pastries': {'baking_skills': 5, 'attention_to_detail': 4}
}
employees = {
    'Alice': {'barista_skills': 1, 'customer_service': 5, 'speed': 4, 'efficiency': 2, 'attention_to_detail': 3, 'organization': 3, 'baking_skills': 2},
    'Bob': {'barista_skills': 4, 'customer_service': 3, 'speed': 5, 'efficiency': 3, 'attention_to_detail': 4, 'organization': 2, 'baking_skills': 3},
    'Charlie': {'barista_skills': 2, 'customer_service': 4, 'speed': 3, 'efficiency': 5, 'attention_to_detail': 5, 'organization': 4, 'baking_skills': 1},
    'David': {'barista_skills': 5, 'customer_service': 2, 'speed': 2, 'efficiency': 4, 'attention_to_detail': 3, 'organization': 5, 'baking_skills': 4},
    'Eve': {'barista_skills': 1, 'customer_service': 4, 'speed': 4, 'efficiency': 5, 'attention_to_detail': 2, 'organization': 3, 'baking_skills': 5}
}


# The distances
locations = ['Helsinki', 'Espoo', 'Tampere', 'Vantaa', 'Oulu', 'Turku']
distances = {
    ('Helsinki', 'Espoo'): 20, ('Helsinki', 'Tampere'): 180,
    ('Helsinki', 'Vantaa'): 15, ('Helsinki', 'Oulu'): 600,
    ('Espoo', 'Tampere'): 170, ('Espoo', 'Vantaa'): 25,
    ('Espoo', 'Oulu'): 610, ('Tampere', 'Vantaa'): 160,
    ('Tampere', 'Oulu'): 420, ('Vantaa', 'Oulu'): 590,
}

# Data for the scheduling problem
shifts = {
    'Morning Shift': '06:00 - 12:00',
    'Afternoon Shift': '12:00 - 18:00',
    'Evening Shift': '18:00 - 00:00'
}
employee_availability = {
    'Alice': ['Morning Shift', 'Afternoon Shift'],
    'Bob': ['Afternoon Shift', 'Evening Shift'],
    'Charlie': ['Morning Shift', 'Evening Shift'],
    'David': ['Afternoon Shift', 'Evening Shift'],
    'Eve': ['Morning Shift', 'Evening Shift']
}

Genetic Algorithm Framework

In [ ]:
def genetic_algorithm(initialize_population_func, fitness_func, mutate_func, pop_size, generations, mutation_rate):
    population = initialize_population_func(pop_size)
    for _ in range(generations):
        fitnesses = [fitness_func(ind) for ind in population]
        population = select(population, fitnesses)
        new_population = []
        while len(new_population) < pop_size:
            parent1, parent2 = random.sample(population, 2)
            offspring1, offspring2 = crossover(parent1, parent2)
            new_population.extend([mutate_func(offspring1, mutation_rate), mutate_func(offspring2, mutation_rate)])
        population = new_population
    return max(population, key=fitness_func)

# Selection function used by the genetic algorithm
def select(population, fitnesses, tournament_size=3):
    selected = []
    for _ in range(len(population)):
        tournament = random.sample(list(zip(population, fitnesses)), tournament_size)
        winner = max(tournament, key=lambda x: x[1])[0]
        selected.append(winner)
    return selected

# Crossover function used by the genetic algorithm
def crossover(parent1, parent2):
    if isinstance(parent1, dict):
        # Dictionary-based crossover (for the assignment problem)
        child1, child2 = parent1.copy(), parent2.copy()
        for task in parent1:
            if random.random() < 0.5:
                child1[task], child2[task] = child2[task], child1[task]
        return child1, child2
    elif isinstance(parent1, list):
        # List-based crossover (for the routing problem)
        return ordered_crossover(parent1, parent2)
    else:
        raise TypeError("Unsupported chromosome type")

def ordered_crossover(parent1, parent2):
    size = len(parent1)
    child1, child2 = [None]*size, [None]*size
    start, end = sorted(random.sample(range(size), 2))
    child1[start:end] = parent1[start:end]
    child2[start:end] = parent2[start:end]
    fill_child(child1, parent2, start, end)
    fill_child(child2, parent1, start, end)
    return child1, child2

def fill_child(child, parent, start, end):
    size = len(parent)
    p_index, c_index = end, end
    while None in child:
        if parent[p_index % size] not in child:
            child[c_index % size] = parent[p_index % size]
            c_index += 1
        p_index += 1

Assignment Problem Specific Functions

In [ ]:
def fitness_assignment(chromosome):
    skill_match_score = 0
    for task, employee in chromosome.items():
        for skill in tasks[task]:
            skill_match_score += min(tasks[task][skill], employees[employee][skill])
    return skill_match_score

def mutate_assignment(chromosome, mutation_rate):
    for task in chromosome:
        if random.random() < mutation_rate:
            chromosome[task] = random.choice(list(employees.keys()))
    return chromosome

def validate_assignment_data(tasks, employees):
    is_valid = True
    for task, skills in tasks.items():
        for skill, level in skills.items():
            if level < 1:
                print(f"Invalid skill level in task '{task}': {skill} level cannot be less than 1.")
                is_valid = False

    for employee, skills in employees.items():
        for skill, level in skills.items():
            if level < 1:
                print(f"Invalid skill level for employee '{employee}': {skill} level cannot be less than 1.")
                is_valid = False

    return is_valid
def initialize_population_assignment(pop_size, tasks, employees):
    if not tasks or not employees:
        print("Invalid input: Both tasks and employees are required for the assignment problem.")
        return []
    return [{task: random.choice(list(employees.keys())) for task in tasks} for _ in range(pop_size)]


Routing Problem Specific Functions

In [ ]:
def calculate_distance(route):
    total_distance = 0
    for i in range(len(route)):
        start, end = route[i], route[(i + 1) % len(route)]  # Wrap around to the start
        total_distance += distances.get((start, end), 0)
    return total_distance

def fitness_routing(chromosome):
    return calculate_distance(chromosome)

def mutate_routing(chromosome, mutation_rate):
    for i in range(len(chromosome)):
        if random.random() < mutation_rate:
            swap_with = random.randint(0, len(chromosome) - 1)
            chromosome[i], chromosome[swap_with] = chromosome[swap_with], chromosome[i]
    return chromosome

# Validation function for the routing problem distances
def validate_distances(distances):
    is_valid = True
    for key, value in distances.items():
        if value <= 0:
            print(f"Invalid distance: Distance between {key[0]} and {key[1]} cannot be 0 or negative.")
            is_valid = False
    return is_valid
# Enhanced initialization for the routing problem to handle edge cases
def initialize_population_routing(pop_size, locations):
    if len(locations) < 2:
        print("Invalid input: At least two locations are required for the routing problem.")
        return []
    return [random.sample(locations, len(locations)) for _ in range(pop_size)]


Scheduling Problem Specific Functions

In [ ]:
def fitness_scheduling(chromosome):
    fitness_score = 0
    for shift, employee in chromosome.items():
        if shift in employee_availability[employee]:
            fitness_score += 1  # Increase fitness for valid assignments
    return fitness_score

def mutate_scheduling(chromosome, mutation_rate):
    for shift in chromosome:
        if random.random() < mutation_rate:
            chromosome[shift] = random.choice(list(employee_availability.keys()))
    return chromosome

def validate_scheduling_data(shifts, employee_availability):
    is_valid = True
    if not shifts:
        print("Invalid input: Shifts are required for the scheduling problem.")
        is_valid = False

    for employee, available_shifts in employee_availability.items():
        for shift in available_shifts:
            if shift not in shifts:
                print(f"Invalid shift '{shift}' in employee '{employee}' availability.")
                is_valid = False

    return is_valid
def initialize_population_scheduling(pop_size, shifts, employee_availability):
    if not shifts or not employee_availability:
        print("Invalid input: Both shifts and employee availability are required for the scheduling problem.")
        return []
    return [{shift: random.choice(list(employee_availability.keys())) for shift in shifts} for _ in range(pop_size)]


Enhanced genetic algorithm function with adjustable parameters

In [ ]:
def run_genetic_algorithm(problem_type, pop_size, generations, mutation_rate, tasks=None, employees=None, locations=None, employee_availability=None):
    # Assignment problem logic
    if problem_type == 'assignment':
        if not validate_assignment_data(tasks, employees):
            print("Assignment data validation failed.")
            return None
        # Capture tasks and employees in the lambda function
        return genetic_algorithm(
            lambda pop_size=pop_size, tasks=tasks, employees=employees: initialize_population_assignment(pop_size, tasks, employees),
            fitness_assignment,
            mutate_assignment,
            pop_size,
            generations,
            mutation_rate
        )
    # Routing problem logic
    elif problem_type == 'routing':
        if not validate_distances(distances):
            print("Routing data validation failed.")
            return None
        return genetic_algorithm(
            lambda pop_size=pop_size, locations=locations: initialize_population_routing(pop_size, locations),
            fitness_routing,
            mutate_routing,
            pop_size,
            generations,
            mutation_rate
        )
    # Scheduling problem logic
    elif problem_type == 'scheduling':
        if not validate_scheduling_data(shifts, employee_availability):
            print("Scheduling data validation failed.")
            return None
        return genetic_algorithm(
            lambda pop_size=pop_size, shifts=shifts, employee_availability=employee_availability: initialize_population_scheduling(pop_size, shifts, employee_availability),
            fitness_scheduling,
            mutate_scheduling,
            pop_size,
            generations,
            mutation_rate
        )

# Execute the genetic algorithm for each problem with different parameter settings
best_assignment = run_genetic_algorithm('assignment', 150, 60, 0.05, tasks=tasks, employees=employees)
best_route = run_genetic_algorithm('routing', 200, 40, 0.08, locations=locations)
best_schedule = run_genetic_algorithm('scheduling', 120, 50, 0.1, tasks=tasks, employee_availability=employee_availability)


Prints

In [ ]:
# Print Best Assignment
print("\n===== Best Assignment =====")
if best_assignment is not None:
    for task, employee in best_assignment.items():
        print(f"  {task} -> assigned to {employee}")
else:
    print("  Error in assignment data. Please check the data section.")

# Print Best Route
print("\n===== Best Route =====")
if best_route is not None:
    route_str = " -> ".join(best_route)
    print(f"  {route_str}")
else:
    print("  Error in routing data. Please check the data section.")

# Print Best Schedule
print("\n===== Best Schedule =====")
if best_schedule is not None:
    for shift, employee in best_schedule.items():
        print(f"  {shift} -> assigned to {employee}")
else:
    print("  Error in scheduling data. Please check the data section.")



===== Best Assignment =====
  Make Coffee -> assigned to Bob
  Serve Customers -> assigned to Alice
  Clean Tables -> assigned to David
  Manage Inventory -> assigned to David
  Bake Pastries -> assigned to Eve

===== Best Route =====
  Turku -> Helsinki -> Tampere -> Vantaa -> Espoo -> Oulu

===== Best Schedule =====
  Morning Shift -> assigned to Alice
  Afternoon Shift -> assigned to Bob
  Evening Shift -> assigned to Eve
